# Análisis Exploratorio de Datos (EDA)

Este cuaderno realiza un EDA básico sobre un dataset hipotético de compras. El análisis se enfoca en preguntas de negocio sencillas y aplica métodos de filtrado y agrupación para calcular métricas relevantes.

In [3]:
!wget https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/shopping_trends.csv

--2025-03-05 22:24:31--  https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/shopping_trends.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453248 (443K) [text/plain]
Saving to: ‘shopping_trends.csv’

shopping_trends.csv 100%[===================>] 442.62K  --.-KB/s    in 0.05s   

2025-03-05 22:24:31 (9.29 MB/s) - ‘shopping_trends.csv’ saved [453248/453248]



## 1. Importar librerías y cargar datos

En este ejemplo, se asume que el archivo CSV se llama `datos_compras.csv`. Ajusta el nombre o la ruta del archivo según corresponda.

In [4]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('shopping_trends.csv')

# Mostrar las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


Observamos que nuestro DataFrame contiene, entre otras, las siguientes columnas:

- **Customer ID**: ID único de cada cliente.
- **Age**: Edad del cliente.
- **Gender**: Género del cliente (Male/Female).
- **Item Purchased**: Nombre del producto comprado.
- **Category**: Categoría del producto (Footwear, Clothing, Accessories, etc.).
- **Purchase Amount (USD)**: Monto de la compra en dólares.
- **Location**: Ubicación/Estado.
- **Size**: Talla del producto.
- **Color**: Color del producto.
- **Season**: Temporada (Summer, Winter, etc.).
- **Review Rating**: Calificación de reseña (escala 1-5).
- **Subscription Status**: Indica si el cliente está suscrito (Yes/No).
- **Shipping Type**: Tipo de envío (Express, Store Pickup, etc.).
- **Discount Applied**: Indica si se aplicó descuento (Yes/No).
- **Promo Code Used**: Indica si se usó código promocional (Yes/No).
- **Previous Purchases**: Número de compras anteriores.
- **Payment Method**: Método de pago (Credit Card, Debit Card, PayPal, etc.).
- **Frequency of Purchases**: Frecuencia de compra (Daily, Weekly, Monthly, etc.).


## 2. Limpieza y revisión rápida de datos
Antes de responder a las preguntas de negocio, revisemos datos básicos: número de filas, columnas y valores nulos.

In [5]:
# Ver información general
df.info ()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Payment Method            3900 non-null   object 
 13  Shipping Type             3900 non-null   object 
 14  Discount

In [6]:
# Ver estadísticos descriptivos de las columnas numéricas
df.describe()


,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.749949,25.351538
std,1125.977353,15.207589,23.685392,0.716223,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.700000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


Si encuentras valores nulos en alguna columna, podrías decidir cómo manejarlos (eliminarlos o rellenarlos). Por simplicidad, asumimos que no hay muchos datos faltantes o que no es crítico para este ejemplo.

## 3. Preguntas de negocio
A continuación, formulamos algunas preguntas de negocio y utilizamos operaciones de filtrado y/o agrupaciones para responderlas.

### 3.1 ¿Cuántos clientes únicos hay?
Esta pregunta nos ayuda a entender la base total de clientes.

In [12]:
num_clientes_unicos = df['Customer ID'].nunique()
print(f"numero de clientes unicos:{num_clientes_unicos}")

numero de clientes unicos:3900


### 3.2 ¿Cuál es el gasto promedio por género?
Para entender si hay diferencias en el monto de compra según el género, hacemos un groupby y calculamos la media.

In [17]:
gasto_promedio_por_genero = df.groupby('Gender')['Purchase Amount (USD)'].mean()
print(gasto_promedio_por_genero)

Gender
Female    60.249199
Male      59.536199
Name: Purchase Amount (USD), dtype: float64


### 3.3 ¿Cuál es la calificación de reseña promedio por temporada?
Si queremos ver si las reseñas cambian según la temporada, podemos hacer una agrupación por la columna `Season` y calcular la media.

In [22]:
Calificacion_proemedio_por_temporada = df.groupby('Season')['Review Rating'].mean()
print(gasto_promedio_por_genero)

Gender
Female    60.249199
Male      59.536199
Name: Purchase Amount (USD), dtype: float64


### 3.4 ¿Cuál es el método de pago más utilizado?
Una forma rápida es usar `value_counts()` en la columna `Payment Method`.

In [24]:
metodo_pago_mas_utlizado = df ['Payment Method'].value_counts().idxmax()
print(f"metodo de pago mas utilizado es:{metodo_pago_mas_utlizado}")

metodo de pago mas utilizado es:Credit Card


### 3.5 ¿Qué categoría se vende más?
Para saber la categoría más popular, también usamos `value_counts()`.

In [27]:
categoria_mas_vendida = df ['Category'].value_counts().idxmax()
print(f"la categoria mas vendida es): {categoria_mas_vendida}")

la categoria mas vendida es): Clothing


### 3.6 ¿Cuántos clientes tienen suscripción activa?
Analizamos la columna `Subscription Status`.

In [30]:
clientes_con_suscripcion = df [df['Subscription Status'] == 'Yes'].shape[0]
print(f"numero de clientes con suscripcion activa:{clientes_con_suscripcion}")

numero de clientes con suscripcion activa:1053


### 3.7 ¿Cuál es el gasto promedio por categoría?
Para entender mejor en qué categoría gastan más nuestros clientes.

In [31]:
gasto_promedio_por_categoria = df.groupby('Category')['Purchase Amount (USD)'].mean()
print(gasto_promedio_por_categoria)

Category
Accessories    59.838710
Clothing       60.025331
Footwear       60.255426
Outerwear      57.172840
Name: Purchase Amount (USD), dtype: float64


### 3.8 ¿Cuántos clientes usan código promocional?
Podemos filtrar o contar los valores de la columna `Promo Code Used`.

In [32]:
clientes_con_codigo_promocional = df[df['Promo Code Used'] == 'Yes'].shape[0]
print(f"numero de clientes con codigo promocional:{clientes_con_codigo_promocional}")

numero de clientes con codigo promocional:1677


## 4. Conclusiones
- **Clientes únicos**: Nos indica cuántas personas diferentes compran en nuestra tienda.
- **Gasto promedio por género**: Sirve para comprender si existe diferencia significativa en el gasto entre hombres y mujeres.
- **Calificación promedio por temporada**: Ayuda a identificar en qué temporada los clientes suelen estar más satisfechos.
- **Métodos de pago preferidos**: Informa sobre la facilidad o preferencia de pago.
- **Categoría más vendida**: Podríamos enfocarnos en promocionarla o analizar por qué se vende más.
- **Suscripciones**: Entender cuántos clientes están suscritos para estrategias de marketing.
- **Uso de códigos promocionales**: Indica si las promociones son efectivas o no.

Este análisis permite identificar patrones y oportunidades para enfocar acciones comerciales y de marketing.